## 학습용 데이터 생성

데이콘에서 라벨링된 데이터 말고도 수많은 원본 코드 샘플을 줬기 때문에 직접 라벨링된 학습용 데이터셋을 만들 수 있다.  
해당 코드는 데이콘의 [미남호일룬](https://dacon.io/myprofile/477473/home)님의 코드를 참고했다.

### Library

In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

random.seed(42)
np.random.seed(42)

### 설정값

In [2]:
path = '.' # train_code 폴더의 바로 위 폴더까지의 경로. 현재 환경에 train_code 폴더가 있다면 '.'이면 되고, 아니라면: ex) C:/.../Code_Similarity/train_code 라면 'C:/.../Code_Similarity'
sample_count = 10000 # 만들고싶은 총 샘플 데이터 수
pair_proportion = 0.5 # 같은 문제를 해결하는 코드끼리 묶여있는 짝의 비율(similar가 1인 샘플의 비율. 0~1사이의 실수)

### 함수 정의

In [3]:
def new_sample_generator(path, sample_count, pair_proportion):
    new_df = pd.DataFrame({'code1_path' : [], 'code2_path' : [], 'code1' : [], 'code2' : [], 'similar' : []})
    pair_count = int(sample_count * pair_proportion)
    not_pair_count = sample_count - pair_count
    sample_similar_list = list(np.random.permutation([1] * pair_count + [0] * not_pair_count))

    problem_num_diction = {}
    for i in range(sample_count):
        boolean = sample_similar_list[i]
        if boolean:
            problem_num_diction[i] = np.random.randint(1, 501)
        else:
            problem_num_diction[i] = list(np.random.choice(range(1, 501), size = 2, replace = False))

    sample_num_diction = {}
    for i in range(sample_count):
        boolean = sample_similar_list[i]
        if boolean:
            sample_num_diction[i] = list(np.random.choice(range(1, 501), size = 2, replace = False))
        else:
            sample_num_diction[i] = list(np.random.choice(range(1, 501), size = 2, replace = True))

    for i in tqdm(range(sample_count)):
        boolean = sample_similar_list[i]
        if boolean:
            code_path_1 = '/train_code/problem{number:03d}/problem{number:03d}_{num}.cpp'.format(number = problem_num_diction[i], num = sample_num_diction[i][0])
            code_path_2 = '/train_code/problem{number:03d}/problem{number:03d}_{num}.cpp'.format(number = problem_num_diction[i], num = sample_num_diction[i][1])
        else:
            code_path_1 = '/train_code/problem{number:03d}/problem{number:03d}_{num}.cpp'.format(number = problem_num_diction[i][0], num = sample_num_diction[i][0])
            code_path_2 = '/train_code/problem{number:03d}/problem{number:03d}_{num}.cpp'.format(number = problem_num_diction[i][1], num = sample_num_diction[i][1])
        code_1 = open(path + code_path_1, encoding='utf-8').read()
        code_2 = open(path + code_path_2, encoding='utf-8').read()
        new_df.loc[i] = ['.' + code_path_1, '.' + code_path_2, code_1, code_2, boolean]
    return new_df

### 데이터 생성

In [4]:
new_df = new_sample_generator(path, sample_count, pair_proportion)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:14<00:00, 671.89it/s]


### 데이터 확인

In [ ]:
new_df

In [ ]:
new_df.info()

### 데이터 저장

In [13]:
# new_df.to_csv('./train_2.csv', index=False)